In [1]:
from sqlalchemy import create_engine
from sqlalchemy.engine import URL
from urllib.parse import unquote

server_name   = "localhost"
database_name = "Bookshop"

connection_string = f"DRIVER=ODBC Driver 17 for SQL Server;SERVER={server_name};DATABASE={database_name};Trusted_Connection=yes"
url_string        = URL.create("mssql+pyodbc", query={"odbc_connect": connection_string})

print('Connecting to database using URL string:')
unquoted_url = unquote(str(url_string))
print(unquoted_url, '\n')

try:    
    engine = create_engine(url_string)
    with engine.connect() as connection:
        print(f'Successfully connected to {database_name}!')
except Exception as e:
    print('Error while connecting to database:\n')
    print(e)

Connecting to database using URL string:
mssql+pyodbc://?odbc_connect=DRIVER=ODBC+Driver+17+for+SQL+Server;SERVER=localhost;DATABASE=Bookshop;Trusted_Connection=yes 

Successfully connected to Bookshop!


In [2]:
from sqlalchemy import text

search = input("Please enter a book title")

print(f"Book Search : {search}\n")

query = """select top 5
	b.Title,
	i.Amount,
	s.Name as 'Shop'

from
	books b
	join Inventory i on b.ISBN13 = i.ISBN13
	join Shops s on i.Shop_Id = s.Shop_Id
where 
    b.Title like '%' + :search_string + '%'"""
    
with engine.connect() as conn:    
    result = conn.execute(text(query),{"search_string" : search} )

    print(f"{'Book Title'.ljust(50)}{'Amount'.ljust(10)}{'Shop'}")
    
    for book in result:
        print(f"{str(book.Title).ljust(50)}{str(book.Amount).ljust(10)}{book.Shop}")

    print(query)

Book Search : The

Book Title                                        Amount    Shop
The Colour Of Magic                               14        Bookworm
The Colour Of Magic                               96        Cosy Reads
The Colour Of Magic                               87        Book Knook
The Light Fantastic                               30        Bookworm
The Light Fantastic                               13        Cosy Reads
select top 5
	b.Title,
	i.Amount,
	s.Name as 'Shop'

from
	books b
	join Inventory i on b.ISBN13 = i.ISBN13
	join Shops s on i.Shop_Id = s.Shop_Id
where 
    b.Title like '%' + :search_string + '%'
